In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections
import copy
import numbers
import os
import sys
import uuid
from itertools import product, combinations
from importlib import reload
sys.path.append(r'C:\dev\projects\weedget')

import ipywidgets as w
import ipyvuetify as v
import ipyvue as vue
import numba
import numpy as np
import pandas as pd
import sklearn
import traitlets

from scipy.special import comb
from plotly import graph_objects as go, express as px
from plotly.subplots import make_subplots
from weedget import base as b, components as c
from macro_features.app import MultiTab

from weedget import transformers
from weedget.transformers import *
from weedget.transformers.pd import *
from weedget.transformers.filters import *
from weedget.transformers.shift import *
from weedget.transformers.outliers import *
from weedget.transformers.np import *
from weedget.transformers.preprocessing import PowerTransformer, Raw, StandardScaler
from weedget.transformers.decomposition import PCA
from weedget.transformers.interaction import *

c:\dev\projects\weedget\venv\lib\site-packages\tigramite\independence_tests\gpdc.py:16: UserWarning: No module named 'dcor'
  warnings.warn(str(e))
c:\dev\projects\weedget\venv\lib\site-packages\tigramite\independence_tests\gpdc_torch.py:18: UserWarning: No module named 'dcor'
  warnings.warn(str(e))


In [3]:
sklearn.set_config(display='diagram')
rng = np.random.default_rng(1988)
pd.options.plotting.backend = "plotly"
pd.options.plotting.backend = "matplotlib"

In [4]:
n_cols = 3
n_rows = 50
start = '1900-01-01'

df = pd.DataFrame(
    data=rng.normal(size=(n_rows, n_cols)),
    index=pd.DatetimeIndex(pd.date_range(start=start, periods=n_rows)),
    # columns=[uuid.uuid4().hex[:10] for i in range(n_cols)]
    columns=[chr(i) for i in range(65, 65 + n_cols)]
)

In [ ]:
features = pd.read_csv('../data/Raw_data.csv', index_col=0, parse_dates=True).drop('US_CBO_Nairu', axis=1).dropna(how='all', axis=1).dropna(how='all')

## Features

### 1. Resampling

In [5]:
resampling = 'W-FRI'
fill = 'ffill'

### 2. Filters

In [6]:
filters = NaNFilter(0.3) + LowVarianceFilter()

### 3. Functions 

In [7]:
functions = union(
    Raw(),
    PowerTransformer(),
)

### 4. Smoothers

In [8]:
smoothers = union(
    Raw(),
    TransformerSpecs(
        factory=EWMZScore,
        params=[
            {'alpha': 0.1},
            {'alpha': 0.2},
            {'alpha': 0.3}
        ],
    ),
    TransformerSpecs(
        factory=RollingZScore,
        params=[
            {'window': 10},
            {'window': 20},
            {'window': 50}
        ],
    )
)

### 5. Difference

In [9]:
difference = union(
    Raw(),
    Difference(),
    Return(),
    TransformerSpecs(
        factory=FracDiff,
        params=[
            {'d': 0.25},
            {'d': 0.5},
            {'d': 0.75}
        ],
    )
)

### 6. Interactions

In [10]:
interaction = Raw() & PairwiseProduct()

### 7. Outliers

In [11]:
clipper = WinsorClipper()

### 8. Final

In [12]:
final = pipe(
    filters,
    functions,
    smoothers,
    filters,
    difference,
#     interaction,
#     clipper,
)
final

Pipeline(steps=[('pipeline-1',
                 Pipeline(steps=[('nanfilter', NaNFilter(n=0.3)),
                                 ('lowvariancefilter', LowVarianceFilter())])),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('raw', Raw()),
                                                ('powertransformer',
                                                 PowerTransformer())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('raw', Raw()),
                                                ('featureunion-1',
                                                 FeatureUnion(transformer_list=[('ewmzscor...
                ('pipeline-2',
                 Pipeline(steps=[('nanfilter', NaNFilter(n=0.3)),
                                 ('lowvariancefilter', LowVarianceFilter())])),
                ('featureunion-3',
                 FeatureUnion(transformer_list=[('raw', Raw()),
                                                ('difference', Difference()),
                                                ('return', Return()),
                                                ('featureunion',
                                                 FeatureUnion(transformer_list=[('fracdiff-1',
                                                                                 FracDiff(d=0.25)),
                                                                                ('fracdiff-2',
                                                                                 FracDiff()),
                                                                                ('fracdiff-3',
                                                                                 FracDiff(d=0.75))]))]))])

In [ ]:
X = getattr(features.resample(resampling).last(), fill)()

In [ ]:
X_tr = final.fit_transform(X)

## Target

In [ ]:
# features.loc[:, features.columns.str.contains(string)]
target_name = 'US_Equity_Fut'
y = X[target_name]

In [ ]:
target_pipe = pipe(
    Log(),
    FracDiff(0.8), 
    PowerTransformer(),
    WinsorClipper()
)
target_pipe

In [ ]:
y_tr = target_pipe.fit_transform(y)